After some tweaking to the lambda api calls, plus based on the experiments I got from v1, now try v2 of analysis. There are many avenues to explore so I will try to stick to locations in this notebook. The inspiration for this part of the analysis is https://datascopeanalytics.com/blog/instagrams-blind-spot/

In [1]:
import numpy as np
import pandas as pd
import pymongo
from pymongo import MongoClient
client = MongoClient()

from pprint import pprint
from bson.son import SON
#for geo parsing
import json
from shapely.geometry import shape, Point
import geopandas as gpd
from shapely.ops import cascaded_union

In [2]:
import datetime
import time

In [3]:
#db setup-- the posts collection is the only one for stashing everything returned from that api endpoint
#note to self-- use mongoDatabase2
db = client['instagram-london']
coll = db.posts

In [4]:
#to illustrate what one of these entries llok like
coll.find_one({'timestamp': {'$gte': datetime.datetime(2016, 9, 5, 0,0,0)}})

{'_id': ObjectId('57f001722285b90b3dd1b8eb'),
 'attribution': None,
 'caption': {'created_time': '1473033605',
  'from': {'full_name': 'R.',
   'id': '2370818488',
   'profile_picture': 'https://scontent.cdninstagram.com/t51.2885-19/s150x150/14073180_1108854822516784_2100955802_a.jpg',
   'username': 'carrerasleon3'},
  'id': '17862635572053336',
  'text': 'Ya hace un tiempo que me siento bien que se que todo lo que pasa va a ponerme okeyyyyyy🤘🏼😝'},
 'comments': {'count': 1},
 'created_time': '1473033605',
 'filter': 'Normal',
 'id': '1332224901537240172_2370818488',
 'images': {'low_resolution': {'height': 320,
   'url': 'https://scontent.cdninstagram.com/t51.2885-15/s320x320/e35/13687353_326294671047465_665291580_n.jpg?ig_cache_key=MTMzMjIyNDkwMTUzNzI0MDE3Mg%3D%3D.2.l',
   'width': 320},
  'standard_resolution': {'height': 640,
   'url': 'https://scontent.cdninstagram.com/t51.2885-15/s640x640/sh0.08/e35/13687353_326294671047465_665291580_n.jpg?ig_cache_key=MTMzMjIyNDkwMTUzNzI0MDE3Mg%

There are quite a few issues with the locations-- for quite a few locations I ahve found that the place have the wrong lat lngs assigned to them.

Some queries that are interesting:

Top tags: (in mongo shell, these are)

```var pipeline = [{$project: {_id: 0, tags: 1}}, {$unwind: "$tags"}, {$group: {_id: "$tags", count: {$sum: 1}}}, {$sort: {count: -1}}, {$limit: 20}]```

Top locations
```var pipeline=[{$group: {_id: {location: '$location'}, count: {$sum: 1}}}, {$sort: {count: -1}}, {$limit: 20}]```

Then run follwing for the aggregated output:

```db.posts.aggregate(pipeline, {allowDiskUse: true})```



In [4]:
#since I changed the range of the lat lngs returned from the lambda function, what I want is all the data after ~ 4th Sept 2016
rawInstagramPosts = coll.find({'timestamp': {'$gte': datetime.datetime(2016, 9, 5, 0,0,0)}})

In [5]:
uniqueLocationsCursor = db.posts.aggregate([
        {"$match": {'timestamp': {'$gte': datetime.datetime(2016, 9, 5, 0,0,0)}}},
        {"$project": {"_id" : 0, "location": 1}},
        {"$group": { "_id": {"location": "$location"}, 
                    "count": { "$sum": 1 }  }},
        {"$sort": {"count" : -1}}
    ]);

uniqueLocations = list(uniqueLocationsCursor)

In [6]:
print("total number of unique locations currently: {}".format(len(uniqueLocations)))

total number of unique locations currently: 79157


In [7]:
uniqueLocations[:10]

[{'_id': {'location': {'id': 213385402,
    'latitude': 51.507114863624,
    'longitude': -0.12731805236353,
    'name': 'London, United Kingdom'}},
  'count': 8524},
 {'_id': {'location': {'id': 230884974,
    'latitude': 51.690636395632,
    'longitude': -0.4187279223942,
    'name': 'Harry Potter Studio Tour at Warner Brothers Studios'}},
  'count': 5802},
 {'_id': {'location': {'id': 594107096,
    'latitude': 51.455329995651,
    'longitude': -0.15493754560226,
    'name': 'Santorini, Greece'}},
  'count': 3904},
 {'_id': {'location': {'id': 21911,
    'latitude': 51.556239704361,
    'longitude': -0.2796649891529,
    'name': 'Wembley Stadium'}},
  'count': 3475},
 {'_id': {'location': {'id': 230734647,
    'latitude': 51.4474504096,
    'longitude': -0.047657510158,
    'name': 'Mykonos, Greece'}},
  'count': 2968},
 {'_id': {'location': {'id': 898717517,
    'latitude': 51.471890719224,
    'longitude': -0.48701449052824,
    'name': 'London Heathrow British Airways Terminal 5'

In [39]:
uniqueLocationsWithLatLng = [x for x in uniqueLocations if ('latitude'  in x['_id']['location'] and 'longitude' in x['_id']['location'])]

In [44]:
londonLocations = [{'name': x['_id']['location']['name'], 
                    'lat': x['_id']['location']['latitude'], 
                    'lng' :  x['_id']['location']['longitude'], 
                    'count': x['count'],
                    'id': x['_id']['location']['id']
                   } 
                   for x in uniqueLocationsWithLatLng]
londonLocations[:100]

[{'count': 7200,
  'id': 213385402,
  'lat': 51.507114863624,
  'lng': -0.12731805236353,
  'name': 'London, United Kingdom'},
 {'count': 4902,
  'id': 230884974,
  'lat': 51.690636395632,
  'lng': -0.4187279223942,
  'name': 'Harry Potter Studio Tour at Warner Brothers Studios'},
 {'count': 3903,
  'id': 594107096,
  'lat': 51.455329995651,
  'lng': -0.15493754560226,
  'name': 'Santorini, Greece'},
 {'count': 2968,
  'id': 230734647,
  'lat': 51.4474504096,
  'lng': -0.047657510158,
  'name': 'Mykonos, Greece'},
 {'count': 2771,
  'id': 21911,
  'lat': 51.556239704361,
  'lng': -0.2796649891529,
  'name': 'Wembley Stadium'},
 {'count': 2479,
  'id': 898717517,
  'lat': 51.471890719224,
  'lng': -0.48701449052824,
  'name': 'London Heathrow British Airways Terminal 5'},
 {'count': 2356,
  'id': 88014,
  'lat': 51.40406242714,
  'lng': -0.33914758033938,
  'name': 'Hampton Court Palace'},
 {'count': 2282,
  'id': 213966560,
  'lat': 51.504327619963,
  'lng': -0.019761837107478,
  'name

There are some issues with the location data-- e.g. in the top 100 locations include Greece, Beijing and Paris!
The 'London' location should also be excluded since that is probably for entries where there is no clear lat lng and defaults to london, which will throw analysis of where posts are taken off.

In [56]:
#placeholder --data cleaning. Basically filter the collection to exclude those with names that include names from http://data.okfn.org/data/core/country-list/r/data.csv
locationIDsToExclude = [213385402,594107096,230734647 ]
londonLocationsFiltered = [x for x in londonLocations if x['id'] not in locationIDsToExclude]

In [49]:
#now use Geopandas to parse the locations into the london census output areas
londonGeoDF = gpd.read_file('OA_2011_BGC_london.json')
#now trim the londonGeo stuff to may it smaller... hopefully this will make any op faster
londonDFSubset = londonGeoDF.ix[:,['OA11CD', 'POPDEN', 'WD11NM_BF', 'geometry']]

In [50]:
#OA11CD is the output area code, popden is population density 
londonDFSubset.head()

,OA11CD,POPDEN,WD11NM_BF,geometry
0,E00000371,85.8,Mayesbrook,"POLYGON ((0.123626833290036 51.54779654745277,..."
1,E00000370,88.3,Mayesbrook,"POLYGON ((0.114355116990793 51.54695588394154,..."
2,E00000372,116.7,Mayesbrook,"POLYGON ((0.120582018615427 51.5468436367351, ..."
3,E00000375,113.7,Mayesbrook,"POLYGON ((0.118107462304139 51.54678149170271,..."
4,E00000377,97.5,Mayesbrook,"POLYGON ((0.120783314586704 51.54480800628029,..."


In [57]:
#using shapely, construct a geodataframe from the uniqueLocations
locationsDF = pd.DataFrame({'geometry': [Point(x['lng'], x['lat']) for x in londonLocationsFiltered], 
                            'count': [x['count'] for x in londonLocationsFiltered],
                            'id': [x['id'] for x in londonLocationsFiltered],
                            'lat': [x['lat'] for x in londonLocationsFiltered],
                            'lng': [x['lng'] for x in londonLocationsFiltered],
                            'name': [x['name'] for x in londonLocationsFiltered]
                           })
locationsDF = gpd.GeoDataFrame(locationsDF)

In [58]:
locationsDF.head()

,count,geometry,id,lat,lng,name
0,4902,POINT (-0.4187279223942 51.690636395632),230884974,51.690636,-0.418728,Harry Potter Studio Tour at Warner Brothers St...
1,2771,POINT (-0.2796649891529 51.556239704361),21911,51.556240,-0.279665,Wembley Stadium
2,2479,POINT (-0.48701449052824 51.471890719224),898717517,51.471891,-0.487014,London Heathrow British Airways Terminal 5
3,2356,POINT (-0.33914758033938 51.40406242714),88014,51.404062,-0.339148,Hampton Court Palace
4,2282,POINT (-0.019761837107478 51.504327619963),213966560,51.504328,-0.019762,London Canary Wharf


In [60]:
#Now merge it with the londonGeoDF using a spatial join to find which OA each location is in
locationsDF.crs = londonDFSubset.crs
mergedLocationDF = gpd.sjoin(locationsDF, londonDFSubset, how="left", op='within')
mergedLocationDF.head()

,count,geometry,id,lat,lng,name,index_right,OA11CD,POPDEN,WD11NM_BF
0,4902,POINT (-0.4187279223942 51.690636395632),230884974,51.690636,-0.418728,Harry Potter Studio Tour at Warner Brothers St...,NaN,NaN,NaN,NaN
1,2771,POINT (-0.2796649891529 51.556239704361),21911,51.556240,-0.279665,Wembley Stadium,2601,E00174722,10.6,Tokyngton
2,2479,POINT (-0.48701449052824 51.471890719224),898717517,51.471891,-0.487014,London Heathrow British Airways Terminal 5,12864,E00012202,1.5,Heathrow Villages
3,2356,POINT (-0.33914758033938 51.40406242714),88014,51.404062,-0.339148,Hampton Court Palace,20164,E00019239,0.9,Hampton
4,2282,POINT (-0.019761837107478 51.504327619963),213966560,51.504328,-0.019762,London Canary Wharf,22045,E00167122,9.1,Millwall


In [61]:
"""next is to groupby the OA11CD field (that should take care of rows where OA11CD is NaN-- this is caused by the way
    the data is gathered (coords defined in a square) -- so there are places which are actually outside london
    E.g. the Warner bros studio is in Levesden which is in Hertfordshire
"""
grouped = mergedLocationDF.groupby('OA11CD').agg({'count': np.sum}).reset_index()
grouped.head()

,OA11CD,count
0,E00000001,2
1,E00000005,1
2,E00000007,23
3,E00000017,14
4,E00000018,71


In [64]:
#just a quick look at the count stats:
grouped.describe()

,count
count,14392.000000
mean,30.934755
std,163.891365
min,1.000000
25%,2.000000
50%,5.000000
75%,16.000000
max,8684.000000


In [67]:
#Now merge it back to the london OA code DF
londonDFSubsetWithCounts = londonDFSubset.merge(grouped, on='OA11CD')

In [71]:
#then get the lat lngs of the centroids of these OA for mapping
londonDFSubsetWithCounts['centroid'] = londonDFSubsetWithCounts['geometry'].centroid
londonDFSubsetWithCounts['lng'] = londonDFSubsetWithCounts['centroid'].map(lambda x: x.xy[0][0])
londonDFSubsetWithCounts['lat'] = londonDFSubsetWithCounts['centroid'].map(lambda x: x.xy[1][0])

In [73]:
londonDFSubsetWithCounts.head()

,OA11CD,POPDEN,WD11NM_BF,geometry,count,centroid,lng,lat
0,E00000373,52.6,Mayesbrook,"POLYGON ((0.112498963564287 51.54358875250732,...",106,POINT (0.1124405823860201 51.54546341002636),0.112441,51.545463
1,E00000374,98.6,Mayesbrook,"POLYGON ((0.116700195832984 51.54488380549373,...",2,POINT (0.1162587508199522 51.54588064596664),0.116259,51.545881
2,E00000363,115.2,Mayesbrook,"POLYGON ((0.122107263387002 51.53850680688704,...",4,POINT (0.1233284720669706 51.53917350591787),0.123328,51.539174
3,E00000476,176.0,Thames,"POLYGON ((0.111826434783545 51.52781877553605,...",4,POINT (0.1106207491860549 51.5284731274997),0.110621,51.528473
4,E00000296,79.1,Goresbrook,"POLYGON ((0.124577913026085 51.53590674281543,...",1,POINT (0.1269292302642004 51.53646909055679),0.126929,51.536469


In [74]:
#now save it to csv:
londonDFSubsetWithCounts.ix[:, ['OA11CD', 'count', 'lng', 'lat', 'POPDEN', 'WD11NM_BF']].to_csv('instagram-london-counts-by-OA.csv')

In [ ]:
#another interesting thing to look at is to count places by hour of day-- e.g. certain locations may be a lot more
#popular by da, while others(e.g. nightclubs) may be the reverse

In [17]:
cursor =  db.posts.find({'$and': [{'timestamp': {'$gte': datetime.datetime(2016, 9, 10, 0,0,0)}},
                         {'timestamp':{'$lt': datetime.datetime(2016, 9, 11, 0, 0, 0)}}, 
                         {'type': "image"}]}, 
               projection=['location','timestamp','likes.count','images.low_resolution','tags'])

In [18]:
oneDayData = pd.DataFrame(list(cursor))

In [19]:
oneDayData[:15]

,_id,images,likes,location,tags,timestamp
0,57d88608d285908f0e1429af,{'low_resolution': {'url': 'https://scontent.c...,{'count': 44},"{'latitude': 51.535331243333, 'id': 228617331,...","[minimal_perfection, jj_indetail, igersoftheda...",2016-09-10 23:59:55
1,57d88608d285908f0e1424e2,{'low_resolution': {'url': 'https://scontent.c...,{'count': 29},"{'latitude': 51.390992716818, 'id': 228474295,...",[],2016-09-10 23:59:50
2,57d885f9d285908f0e13c821,{'low_resolution': {'url': 'https://scontent.c...,{'count': 19},"{'latitude': 51.567947692372, 'id': 607397483,...",[],2016-09-10 23:59:46
3,57d885f5d285908f0e13a7d0,{'low_resolution': {'url': 'https://scontent.c...,{'count': 4},"{'latitude': 51.49525, 'id': 536879592, 'longi...",[],2016-09-10 23:59:36
4,57d88602d285908f0e1401d2,{'low_resolution': {'url': 'https://scontent.c...,{'count': 1},"{'latitude': 51.583, 'id': 227127780, 'longitu...","[praisdentalcare, colgatebrightsmile, colgateb...",2016-09-10 23:59:33
5,57d88601d285908f0e13fa28,{'low_resolution': {'url': 'https://scontent.c...,{'count': 18},"{'latitude': 51.351399783335, 'id': 518090644,...","[londonliving, streetphotography, love, nighto...",2016-09-10 23:59:08
6,57d88608d285908f0e14292b,{'low_resolution': {'url': 'https://scontent.c...,{'count': 3},"{'latitude': 51.517353421125, 'id': 3854922, '...","[djs, popup, foodie, nw10, platta, music]",2016-09-10 23:59:08
7,57d885fad285908f0e13c9b2,{'low_resolution': {'url': 'https://scontent.c...,{'count': 10},"{'latitude': 51.6166915, 'id': 694474516, 'lon...","[nomakeup, nofilter, needsomemakeupforfil, lov...",2016-09-10 23:59:07
8,57d88606d285908f0e141a2b,{'low_resolution': {'url': 'https://scontent.c...,{'count': 14},"{'latitude': 51.492788433808, 'id': 494363078,...","[saturdaynight, awesome, drink, tbt, partytime...",2016-09-10 23:58:56
9,57d885f7d285908f0e13b39e,{'low_resolution': {'url': 'https://scontent.c...,{'count': 20},"{'latitude': 51.42889839, 'id': 295237448, 'lo...","[crazygirl, thropepark, london]",2016-09-10 23:58:52


In [32]:
#a few bits of data cleaning
oneDayData['likes'] = oneDayData['likes'].map(lambda x: x['count'])
oneDayData['locationID'] = oneDayData['location'].map(lambda x: str(x['id']))
oneDayData['lat'] = oneDayData['location'].map(lambda x: x['latitude'])
oneDayData['lng'] = oneDayData['location'].map(lambda x: x['longitude'])
oneDayData['hourOfDay'] = oneDayData['timestamp'].map(lambda x: x.hour)

In [33]:
oneDayData[:15]

,_id,images,likes,location,tags,timestamp,locationID,lat,lng,hourOfDay
0,57d88608d285908f0e1429af,{'low_resolution': {'url': 'https://scontent.c...,44,"{'latitude': 51.535331243333, 'id': 228617331,...","[minimal_perfection, jj_indetail, igersoftheda...",2016-09-10 23:59:55,228617331,51.535331,-0.239905,23
1,57d88608d285908f0e1424e2,{'low_resolution': {'url': 'https://scontent.c...,29,"{'latitude': 51.390992716818, 'id': 228474295,...",[],2016-09-10 23:59:50,228474295,51.390993,-0.318341,23
2,57d885f9d285908f0e13c821,{'low_resolution': {'url': 'https://scontent.c...,19,"{'latitude': 51.567947692372, 'id': 607397483,...",[],2016-09-10 23:59:46,607397483,51.567948,0.140318,23
3,57d885f5d285908f0e13a7d0,{'low_resolution': {'url': 'https://scontent.c...,4,"{'latitude': 51.49525, 'id': 536879592, 'longi...",[],2016-09-10 23:59:36,536879592,51.495250,0.116060,23
4,57d88602d285908f0e1401d2,{'low_resolution': {'url': 'https://scontent.c...,1,"{'latitude': 51.583, 'id': 227127780, 'longitu...","[praisdentalcare, colgatebrightsmile, colgateb...",2016-09-10 23:59:33,227127780,51.583000,-0.185000,23
5,57d88601d285908f0e13fa28,{'low_resolution': {'url': 'https://scontent.c...,18,"{'latitude': 51.351399783335, 'id': 518090644,...","[londonliving, streetphotography, love, nighto...",2016-09-10 23:59:08,518090644,51.351400,-0.169908,23
6,57d88608d285908f0e14292b,{'low_resolution': {'url': 'https://scontent.c...,3,"{'latitude': 51.517353421125, 'id': 3854922, '...","[djs, popup, foodie, nw10, platta, music]",2016-09-10 23:59:08,3854922,51.517353,-0.250053,23
7,57d885fad285908f0e13c9b2,{'low_resolution': {'url': 'https://scontent.c...,10,"{'latitude': 51.6166915, 'id': 694474516, 'lon...","[nomakeup, nofilter, needsomemakeupforfil, lov...",2016-09-10 23:59:07,694474516,51.616692,-0.000394,23
8,57d88606d285908f0e141a2b,{'low_resolution': {'url': 'https://scontent.c...,14,"{'latitude': 51.492788433808, 'id': 494363078,...","[saturdaynight, awesome, drink, tbt, partytime...",2016-09-10 23:58:56,494363078,51.492788,-0.224223,23
9,57d885f7d285908f0e13b39e,{'low_resolution': {'url': 'https://scontent.c...,20,"{'latitude': 51.42889839, 'id': 295237448, 'lo...","[crazygirl, thropepark, london]",2016-09-10 23:58:52,295237448,51.428898,-0.509624,23


In [39]:
oneDayDataGrouped = oneDayData.groupby(['hourOfDay', 'locationID']).size().reset_index(name='count')

In [40]:
oneDayDataGrouped

,hourOfDay,locationID,count
0,0,10001,5
1,0,1011201341,1
2,0,1012587439,1
3,0,1015934491,1
4,0,1016110338,1
5,0,1017396848,1
6,0,1018660003,1
7,0,1019560362,1
8,0,1020910452,1
9,0,1023814140,1
